# Text processing with an RNN  
  
In order to get the RNN to consume text data, we need two things:  
  
1. A list of all the texts, eg `texts = ['Today is a nice day', 'Yesterday was gorgeous',....]`  
2. We need the labels in a list too, eg `labels = [1, 0, 1, 1,...]`.  If labels are text string categories, then use one-hot encoding to convert them into an array, with each row of the array corresponding to an observation.  
  
We first set up a tokenizer as follows:  
  
```python
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences  
  
tokenizer = Tokenizer(num_words = vocab_size) #Sets up a blank tokenizer  
tokenizer.fit_on_texts(texts)   
# fit_on_texts creates the vocabulary based on `texts`, based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value. 0 is reserved for padding. So lower integer means more frequent word (often the first few are stop words because they appear a lot).  
  
sequences = tokenizer.texts_to_sequences(texts)  
# This step transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary.   
  
word_index = tokenizer.word_index # This is the dictionary of words and their indexes which one can see using  word_index.items()  
  
data = pad_sequences(sequences, maxlen = maxlen) # Here we make all texts equal in length, and padding is pre (can be changed   by padding='post')  
  
labels = np.asarray(labels) # We convert the labels into an array from its original list form.  

```  
  
At this point all our input text data exists as perfectly sized tensors in the variable called `data`, and `labels`.  The `data` variable has references to the `word_index` which is essentially the dictionary for the task.  
  
Next, we need to create the embeddings_matrix based on Glove.  For this, we first load up Glove in a giant matrix, and then we look up each item in the `word_index` in the Glove matrix and create our pre-trained embeddings_matrix based on Glove.  In this matrix, we preserve the index order as in the `word_index`, so what is #3 in the word index is also the 3rd row in the embeddings_matrix.  The 0th row is all zeros, and only the first row onwards are used because in the `word_index` dictionary there is no entry for zero.  
  
Now we are good to go to feed this into our NN. The pretrained embeddings_matrix makes its way into the NN using the following commands:  
  
```python
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
```
  
  

In [1]:
import os
os.getcwd()

'C:\\Users\\user'

In [2]:
import os
imdb_dir = '\\Users\\user\\aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels=[]
texts=[]



In [3]:
# Read all the text files
for label_type in ['neg','pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:]== '.txt':
            f = open(os.path.join(dir_name, fname))
            #print(fname)
            texts.append(f.read())
            f.close()
            if label_type =='neg':
                labels.append(0)
            else:
                labels.append(1)
            
        


At this point, the variable `texts` contains all the reviews in plaintext as a list.

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen=500
training_samples=20000
validation_samples = 10000
vocab_size=10000

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))
data = pad_sequences(sequences, maxlen = maxlen)
labels = np.asarray(labels)
print('Shape of data tensor', data.shape)
print('Shape of label tensor', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:training_samples+validation_samples]
y_val = labels[training_samples:training_samples+validation_samples]



Using TensorFlow backend.


Found 88588 unique tokens
Shape of data tensor (25000, 500)
Shape of label tensor (25000,)


In [5]:
len(data)

25000

In [6]:
glove_dir = '\\Users\\user\\glove.6B'

embeddings_index = {}
#f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
f=open('\\Users\\user\\glove.6B\\glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s words and corresponding vectors' % len(embeddings_index))

Found 400000 words and corresponding vectors


In [7]:
embeddings_index.get('the')

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [8]:
print(embeddings_index.get('th13e'))

None


In [9]:
# Create the embedding matrix based on Glove
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            

In [10]:
embedding_matrix.shape

(10000, 100)

At this point the `embedding_matrix` has one row per word in the vocabulary.  Each row has the vector for that word, picked from glove.  Because it is an np.array, it has no row or column names. The order of the words in the rows is the same as the order of words in the dict word_index.  
  
We will feed this embedding matrix as weights to the embedding layer.  
  
## Build the model  

In [11]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, SimpleRNN


In [12]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim)) # Note that vocab_size=10000 (vocab size), embedding_dim = 100 (100 dense vector for each word from Glove), maxlen=100 (using only first 100 words of each review)
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,017,057
Trainable params: 1,017,057
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=15, batch_size=128, validation_split=0.2)

Train on 16000 samples, validate on 4000 samples
Epoch 1/15
16000/16000 [==============================] - 159s 10ms/step - loss: 0.3427 - acc: 0.8536 - val_loss: 0.3584 - val_acc: 0.8438
Epoch 2/15
16000/16000 [==============================] - 168s 10ms/step - loss: 0.3253 - acc: 0.8601 - val_loss: 0.3589 - val_acc: 0.8393
Epoch 3/15
16000/16000 [==============================] - 168s 10ms/step - loss: 0.3148 - acc: 0.8673 - val_loss: 0.3514 - val_acc: 0.8450
Epoch 4/15
16000/16000 [==============================] - 172s 11ms/step - loss: 0.3071 - acc: 0.8708 - val_loss: 0.3423 - val_acc: 0.8540
Epoch 5/15
16000/16000 [==============================] - 181s 11ms/step - loss: 0.2994 - acc: 0.8742 - val_loss: 0.3485 - val_acc: 0.8540
Epoch 6/15
16000/16000 [==============================] - 170s 11ms/step - loss: 0.2894 - acc: 0.8796 - val_loss: 0.3447 - val_acc: 0.8593
Epoch 7/15
16000/16000 [==============================] - 167s 10ms/step - loss: 0.2814 - acc: 0.8829 - val_loss: 0.3